## 自动 生成文章的摘要
使用word2vec+textrank方法的基本流程：
1.将文章分句转换成一个链表
2.将每个句子进行分词，去掉标点符号，去掉停用词，得到一个二维列表，一维表示一句话，一维表示一句话中的词语
3.将句子与其他的句子进行两两相似度。将句子中的每个词的词向量相加平均，再通过句子向量的余玄相似度，得出两个句子的相似度。
  （这里对于在词向量模型中未出现的词做了移除）
4.计算每一句相对于另外的所有句子的得分类似于pagerank的算法。需要一个二维列表存取每一个句子相对于其他句子的得分。
5.迭代计算每一句的分数，重复迭代，直到分数的差值在0.0001下。
6.取出得分最高的topN条数据

In [1]:
import jieba
import math 
from string import punctuation 
from heapq import nlargest 
from itertools import product, count 
from gensim.models import word2vec 
import numpy as np


In [13]:
model = word2vec.Word2Vec.load("G:/github/lesson4/model/wiki_corpus.model")
np.seterr(all='warn')

s = model.similarity("孙悟空", "猪八戒")

print("-----------",s)


g:\Anaconda\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


----------- 0.78556645


C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.


In [20]:
import re
import jieba


def cut(string):
    return ' '.join(jieba.cut(string))


def token(string):
    string = re.findall('[\d|\w|\u3002 |\uff1f |\uff01 |\uff0c |\u3001 |\uff1b |\uff1a |\u201c |\u201d |\u2018 |\u2019 |\uff08 |\uff09 |\u300a |\u300b |\u3008 |\u3009 |\u3010 |\u3011 |\u300e |\u300f |\u300c |\u300d |\ufe43 |\ufe44 |\u3014 |\u3015 |\u2026 |\u2014 |\uff5e |\ufe4f |\uffe5]+', string)
    return ' '.join(string)


def deal(string):
    string = token(string)
    return cut(string)


In [51]:
def cut_sentences(sentence):    
    puns = frozenset(u'。！？')    
    tmp = []    
    for ch in sentence:        
        tmp.append(ch)        
        if puns.__contains__(ch):            
            yield ''.join(tmp)            
            tmp = []    
    yield ''.join(tmp)  
# 句子中的stopwords
def create_stopwords():    
     stop_list = [line.strip() for line in open("G:/github/lesson4/stop_words/stopwords.txt", 'r', encoding='utf-8').readlines()]   
     return stop_list  
def two_sentences_similarity(sents_1, sents_2):    
    '''    计算两个句子的相似性    :
    param sents_1:    
    :param sents_2:   
    :return:    
    '''    
    counter = 0    
    for sent in sents_1:        
        if sent in sents_2:            
            counter += 1    
    return counter / (math.log(len(sents_1) + len(sents_2)))  
def create_graph(word_sent):    
    """    传入句子链表  返回句子之间相似度的图    :param word_sent:    :return:    """    
    num = len(word_sent)    
    board = [[0.0 for _ in range(num)] for _ in range(num)]     
    for i, j in product(range(num), repeat=2):        
        if i != j:            
            board[i][j] = compute_similarity_by_avg(word_sent[i], word_sent[j])    
    return board 
def cosine_similarity(vec1, vec2):   
        '' '    计算两个向量之间的余弦相似度    :param vec1:    :param vec2:    :return:    '''    
        tx = np.array(vec1)    
        ty = np.array(vec2)    
        cos1 = np.sum(tx * ty)    
        cos21 = np.sqrt(sum(tx ** 2))    
        cos22 = np.sqrt(sum(ty ** 2))    
        cosine_value = cos1 / float(cos21 * cos22)   
        return cosine_value  
def compute_similarity_by_avg(sents_1, sents_2):   
    '''    对两个句子求平均词向量    :param sents_1:    :param sents_2:    :return:    '''    
    if len(sents_1) == 0 or len(sents_2) == 0:       
        return 0.0    
    vec1 = model[sents_1[0]]    
    for word1 in sents_1[1:]:
       # print("word1=========",word1)
        vec1 = vec1 + model[word1]     
    vec2 = model[sents_2[0]]    
    for word2 in sents_2[1:]:       
        vec2 = vec2 + model[word2]     
    similarity = cosine_similarity(vec1 / len(sents_1), vec2 / len(sents_2))    
    return similarity  
def calculate_score(weight_graph, scores, i):    
    """    计算句子在图中的分数    :param weight_graph:    :param scores:    :param i:    :return:    """    
    length = len(weight_graph)    
    d = 0.85    
    added_score = 0.0     
    for j in range(length):        
        fraction = 0.0        
        denominator = 0.0        
        # 计算分子        
        fraction = weight_graph[j][i] * scores[j]       
        # 计算分母        
        for k in range(length):            
            denominator += weight_graph[j][k]            
            if denominator == 0:                
                denominator = 1        
        added_score += fraction / denominator    
    # 算出最终的分数   
    weighted_score = (1 - d) + d * added_score    
    return weighted_score 
def weight_sentences_rank(weight_graph):   
    '''    输入相似度的图（矩阵)    返回各个句子的分数    :param weight_graph:    :return:    '''   
    # 初始分数设置为0.5    
    scores = [0.5 for _ in range(len(weight_graph))]    
    old_scores = [0.0 for _ in range(len(weight_graph))]     
    # 开始迭代    
    while different(scores, old_scores):        
        for i in range(len(weight_graph)):            
            old_scores[i] = scores[i]        
        for i in range(len(weight_graph)):            
            scores[i] = calculate_score(weight_graph, scores, i)    
    return scores  
def different(scores, old_scores):   
    '''    判断前后分数有无变化    :param scores:    :param old_scores:    :return:    '''    
    flag = False    
    for i in range(len(scores)):        
        if math.fabs(scores[i] - old_scores[i]) >= 0.0001:           
            flag = True            
            break    
    return flag  
def filter_symbols(sents):    
    stopwords = create_stopwords() + ['。', ' ', '.']    
    _sents = []    
    for sentence in sents:
        s = []
        for word in sentence:            
            if word not in stopwords: 
                #print("word=====",word)
                #sentence.remove(word) 
                s.append(word)
        if len(s)> 0:             
            _sents.append(s)    
    return _sents  
def filter_model(sents):    
    _sents = []    
    for sentence in sents: 
        s = []
        for word in sentence:            
            if word not in model:  
                #print("word=====",word)
                sentence.remove(word)
            else:
                s.append(word)
        if len(s) > 0:            
            _sents.append(s)    
    return _sents  
def summarize(text, n):    
    tokens = cut_sentences(text)    
    sentences = []    
    sents = []    
    for sent in tokens:        
        sentences.append(sent)          
        #sents.append([word for word in jieba.cut(sent) if word])  
        sents.append(deal(sent).split())
    sents = filter_symbols(sents)  
    
    sents = filter_model(sents)    
    #print(sents)
    graph = create_graph(sents)     
    scores = weight_sentences_rank(graph)    
    sent_selected = nlargest(n, zip(scores, count()))    
    sent_index = []    
    for i in range(n):        
        sent_index.append(sent_selected[i][1])   
    return [sentences[i] for i in sent_index]

In [52]:
string = "党的十八大以来，以习近平同志为核心的党中央团结带领全国各族人民坚定不移走中国特色社会主义政治发展道路，坚持党的领导、人民当家作主、依法治国有机统一，推动人大工作取得历史性成就，人民代表大会制度持续创新发展，展现出蓬勃生机和活力。党的十八大以来人民代表大会制度有哪些重要实践创新成果？第5期《求是》杂志刊发中共中央政治局委员、全国人大常委会副委员长王晨同志文章《新时代坚持和完善人民代表大会制度的根本遵循——学习贯彻习近平总书记关于坚持和完善人民代表大会制度的重要思想》对此给出了答案，带您进行全面了解。"

In [58]:
string = "以前，我总是向病人提及惯常推荐的运动时长（即每周低强度运动150分钟或高强度运动90分钟），但现在我可以将运动形容得更有吸引力——15分钟一次，每周3次。当然，这对于已经在坚持健身或参加体育运动的人来说，并无吸引力。而且，关于高强度间歇性锻炼，目前还缺少长期效果和风险减少方面的数据。但对于像我本人这样几乎不参加体育活动的“沉默的大多数”而言，进行高强度间歇性锻炼很显然比什么都不做要好很多，而且从全球范围来看，它可以挽救数以百万计的生命"

In [59]:
print("str==",string)
print(summarize(string , 1))

str== 以前，我总是向病人提及惯常推荐的运动时长（即每周低强度运动150分钟或高强度运动90分钟），但现在我可以将运动形容得更有吸引力——15分钟一次，每周3次。当然，这对于已经在坚持健身或参加体育运动的人来说，并无吸引力。而且，关于高强度间歇性锻炼，目前还缺少长期效果和风险减少方面的数据。但对于像我本人这样几乎不参加体育活动的“沉默的大多数”而言，进行高强度间歇性锻炼很显然比什么都不做要好很多，而且从全球范围来看，它可以挽救数以百万计的生命
['但对于像我本人这样几乎不参加体育活动的“沉默的大多数”而言，进行高强度间歇性锻炼很显然比什么都不做要好很多，而且从全球范围来看，它可以挽救数以百万计的生命']


C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:112: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:49: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `__getitem__` (Method will

In [60]:
string = "5日凌晨，香港警方紧急召开记者会通报称，全港多区4日入夜后出现暴徒纵火、投掷汽油弹、打砸店铺并殴打市民等严重暴力事件。面对这一情况，全港多区超过20个大型购物中心宣布周六停业。港铁则宣布全部地铁、机场快线和港铁巴士于5日早上全线停运。此外，亦有多家中资银行宣布周六暂停服务，与此同时全港多所大学宣布5日全日停课。香港特区政府引用《紧急法》订立的《禁止蒙面规例》，5日零时开始生效。5日凌晨，香港警方紧急召开记者会通报称，全港多区4日入夜后出现暴徒纵火、投掷汽油弹、打砸店铺并殴打市民等严重暴力事件。面对这一情况，为防5日白天出现风险，全港多区超过20个大型购物中心宣布周六停业。港铁则宣布，全部地铁、机场快线和港铁巴士于5日早上全线停运。此外，亦有多家中资银行宣布周六暂停服务。香港警方5日凌晨表示，严重暴力行为在短时间内迅速蔓延到全港14个区。一些商店被严重破坏，货物散落一地。有暴徒如劫匪一般撬开银行闸门，在里面纵火。更有大批暴徒围攻一名便衣警察，该警员因自卫而开了一枪，期间暴徒甚至试图抢枪。“昨晚香港经历了二次大战以来最大规模的破坏，今早港铁全线瘫痪，没有任何其他城市像昨晚的香港如此被自己的居民强暴。”香港前特首梁振英5日这样在社交账号上发帖感叹道。5日上午，环球时报记者在香港看到，许多大型购物中心当日都没有开放，并在大门或墙壁上贴上停业通知。据了解，周六停业的大型商场包括九龙湾德福广场及淘大商场、沙田连城广场及新城市广场、观塘apm商场、将军澳商场PopCorn等。记者同时看到，除大型商场关闭外，香港许多街边的小型商铺当天也落下铁闸，闭门不出.与此同时，全港多所大学宣布5日全日停课，包括香港大学、浸会大学、珠海学院、岭南大学、公开大学及理工大学。有学生向环球时报记者证实，自己所在的大学从前一日下午和晚上就紧急封闭图书馆、篮球场、自习室等公共地点，许多中小学周五亦提前放学，让学生尽早回家。"

In [61]:
print(summarize(string , 2))

C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:112: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:49: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `__getitem__` (Method will

['5日上午，环球时报记者在香港看到，许多大型购物中心当日都没有开放，并在大门或墙壁上贴上停业通知。', '5日凌晨，香港警方紧急召开记者会通报称，全港多区4日入夜后出现暴徒纵火、投掷汽油弹、打砸店铺并殴打市民等严重暴力事件。']


In [62]:
string = "10月1日上午，庆祝中华人民共和国成立70周年大会在北京天安门广场隆重举行，20余万军民以盛大的阅兵仪式和群众游行欢庆共和国七十华诞。中共中央总书记、国家主席、中央军委主席习近平发表重要讲话并检阅受阅部队。国外专家学者认为，习近平主席的重要讲话再次宣示，发展起来的中国将坚持和平发展道路，继续同世界各国人民一道推动共建人类命运共同体。中国取得的伟大成就鼓舞着世界上其他发展中国家。中国的发展惠及全世界俄罗斯人民友谊大学教授尤里·塔夫罗夫斯基表示：中国共产党的领导是70年来中国取得举世瞩目发展成就的根本保证。得益于中国共产党的领导，中国能够从整体角度和长远利益出发考虑发展道路塔夫罗夫斯基深信中国未来将取得更大的成就。德国联邦参议院议长、石勒苏益格—荷尔斯泰因州州长君特表示，中国过去70年的发展速度令人印象深刻，我对中国人民取得的成就感到钦佩。他期待德中两国携手前行，加强合作。德国汉堡市市长彼得·辰切尔表示70年来，中国发生了天翻地覆的变化，发展成为高度开放、与时俱进的现代化国家。汉堡愿意成为共建一带一路的伙伴。比利时荷兰语《今日中国》杂志社总编辑丽娜·登格鲁丹伊森说，中国在过去70年里创造了许多奇迹，让世界赞叹。贫困是世界性难题。中国共产党成功带领数亿人民摆脱贫困，这样的减贫成绩堪称人类历史上的壮举，充分体现了中国共产党以民为本、以人为本的执政理念。日本武藏野大学教授加藤青延表示，70年间，中国从贫穷落后发展成为世界第二大经济体，有着重要国际影响力。相信在中国共产党的领导下，中国现代化建设将取得更加辉煌的成就。巴基斯坦独立新闻社中国业务总监贾韦德·阿赫塔尔说，中国70年来的发展成就堪称奇迹。中国的和平发展为全世界带来巨大机遇，为维护世界和平与稳定作出贡献。南非约翰内斯堡大学非洲—中国研究中心主任戴维·蒙亚埃表示新中国70年的历史经验告诉我们，发展是解决问题的基础和关键。肯尼亚经济学家姆旺吉·瓦吉拉表示，70年来，在中国共产党的领导下，中国成就有目共睹。许多非洲国家都希望学习和借鉴中国的发展经验。习近平主席的讲话强调中国将继续与世界各国人民推动共建人类命运共同体，这也是各国人民的共同愿望。埃及《消息报》总编辑亚西尔·里兹格说，习近平主席的讲话以及国庆阅兵向世界展示了中国的成就与能力，宣示了中国坚定捍卫国家主权的决心。中国这些成就的取得，是坚持共产党领导、坚持中国特色社会主义道路的必然结果。中国的发展惠及全世界，中国将继续同世界各国人民一道推动共建人类命运共同体。中国是维护世界和平的重要力量俄罗斯军事科学院院士、俄罗斯科学院远东研究所首席研究员弗拉基米尔·彼得罗夫斯基表示，中国的国庆阅兵气势恢宏，令人震撼。中国一直是世界和平的坚定维护者，盛大阅兵展现了中国维护世界和平的能力和信心。布鲁塞尔咨询机构第一政策分析师伊格·萨基思说，中华人民共和国成立70年来取得的成就是全方位的。这是中国人民努力拼搏的结果。习近平主席在重要讲话中向外界传递了明确信号，中国将继续是维护世界和平、促进共同发展的重要力量。法国历史学家、作家伯尔纳·布里泽表示，盛大阅兵展现了中国的军事实力和中国军人的精神风貌。中国积极参与国际事务，对维护世界和平和地区稳定作出了贡献。无论是联合国维和任务还是国际救援项目，中国都积极参与、出色完成，展现出大国责任与担当。曾担任美国广播公司电视网络副总裁的哈维·朝鼎表示，中国在短短几十年的时间里，走过了发达国家几百年的历程。中国向世界表明，国际关系不是零和博弈，应当建立在共建人类命运共同体的基础之上。马来西亚执政联盟成员人民公正党副主席郑立慷表示，和平与发展是全世界的共同期待，习近平主席强调中国坚持和平发展道路、奉行互利共赢的开放战略，值得全世界赞赏。柬埔寨王家研究院孔子学院前任柬方院长谢莫尼勒表示，70年来，中国在各领域取得了伟大成就。中国一直坚持国家不分大小、强弱、贫富一律平等。中国坚持和平发展道路，已经并将继续为世界的和平、稳定和繁荣作出积极贡献。巴西外交部大使爱德华多·拉莫斯表示，天安门广场举行的阅兵仪式上，维和部队方队格外引人注目。爱好和平是中国的传统，中国是促进世界和平的稳定器。事实表明，中国是维护世界和平的重要力量联合国马里多层面稳定特派团(联马团)司令丹尼斯·吉伦斯伯尔中将表示，中国是世界和平的重要维护者。他说，目前有421名中国维和人员在联马团执行警卫、工程、医疗等任务，拥有来自中国的力量是维和部队的骄傲。目前正和中国维和部队一起在马里执行维和任务的孟加拉国步兵营上尉军官里亚兹说，阅兵式上中国军队装备先进，中国军人精神昂扬，展现了中国强大的国防力量和中国军人的优良作风。"

In [63]:
print(summarize(string , 2))

C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:112: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:49: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\lenovo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `__getitem__` (Method will

['国外专家学者认为，习近平主席的重要讲话再次宣示，发展起来的中国将坚持和平发展道路，继续同世界各国人民一道推动共建人类命运共同体。', '中国的发展惠及全世界，中国将继续同世界各国人民一道推动共建人类命运共同体。']
